In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(solution):
    sum_gain = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        gain = res[1]-res[2]
        sum_gain += gain
    return (sum_gain,)

In [4]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [5]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [6]:
def genetic_algorithm(initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats
    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-initial_range,initial_range)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = toolbox.population(n=pop_size)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,pop)):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,invalid_ind)):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)

    return hof,logbook

In [7]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [8]:
def objective(trial):
    
    params = {'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 20,20,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(3):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['Max'].mean()


In [11]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=100)

[I 2023-10-13 16:52:59,403] A new study created in memory with name: GA
Process ForkPoolWorker-242:
Process ForkPoolWorker-244:
Process ForkPoolWorker-243:
Process ForkPoolWorker-241:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwarg

KeyboardInterrupt: 

In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""